# Settings

In [24]:
import sys
import os

os.chdir('..')
os.getcwd()

'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub'

# Imports

In [25]:
from scripts.model import CharBiLSTM, prepare_data
from scripts.data import create_data_loader, load_data
from scripts.preprocessing import get_typoglycemia_modified_data, sentence_tokennizer, tokenize_dataframe, get_max_length
from scripts.baseline import get_base_line_score

from torch import nn, optim
import torch

from sklearn.model_selection import train_test_split

import editdistance
from tqdm.autonotebook import trange, tqdm
import random 

from sklearn.metrics import f1_score, accuracy_score
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

# Hard

In [26]:
from model_train_easy import device, complexity_level, max_length, batch_size, input_size, hidden_size, output_size, num_layers, complexity_level, model, loss_function, optimizer, epochs

In [27]:
device, complexity_level

(device(type='cuda', index=0), 'Easy')

In [28]:
train_loader, validation_loader, test_loader = prepare_data(complexity_level = complexity_level,
                                                            max_length = max_length,
                                                            batch_size = batch_size)


KeyboardInterrupt



In [ ]:
for sample in train_loader:
    X, y = sample
    print(X.shape, y.shape)
    print(np.unique(y.cpu(), return_counts=True))
    print(X.device)
    break

torch.Size([512, 150]) torch.Size([512, 150])
(array([-1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26], dtype=int64), array([42620,  2678,   481,   906,  1049,  3331,   633,   519,  1435,
        2236,    99,   215,  1187,   741,  2139,  2026,   591,    35,
        1931,  1878,  2414,   736,   248,   496,    40,   502,  5634],
      dtype=int64))
cpu


In [ ]:
# PATH = f'models/{complexity_level.lower()}/model_{epochs}.pt'
PATH = "models/model_182.pt"
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

CharBiLSTM(
  (lstm): LSTM(1, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=28, bias=True)
)

In [ ]:
def index_to_char(index, target = True):
    if target == False:
        index = round(index*100, 0)
    index = int(index)
    if 1 <= index <= 25:
        return chr(index + ord('a') - 1), index
    elif index == 26:
        return ' ', index
    else:
        return "_", index  # for all other values, return None

In [62]:
import csv
import editdistance

def get_metrics(loader, model, loader_str, output_file="output.csv", sample_size=10):
    preds = []
    labels = []
    inputs = []  # To store input sequences
    
    model.eval()
    loss = 0.0
    csv_data = [["Input Word", "Label Word", "Predicted Word"]]  # CSV header
    sentence_label = []
    sentence_pred = []
    sentence_input = []  # To store input words
    

    with torch.no_grad():
        for batch in tqdm(loader, position=3, leave=False):
            typo_batch, sentence_batch = batch  # typo_batch is the input, sentence_batch is the target sequence
            typo_batch = typo_batch.to(device)
            sentence_batch =sentence_batch.to(device)
            
            sentence_batch = sentence_batch.reshape(-1, max_length, 1)
            typo_batch = typo_batch.reshape(-1, max_length, 1)
            y = model.forward(typo_batch, train=False)

            
            sentence_batch_loss = sentence_batch.reshape(-1)
            # Calculate and accumulate loss
            loss_batch = loss_function(y, sentence_batch_loss)
            loss += loss_batch.item()
            
            

            word_label = []
            word_pred = []
            word_input = []
            
            y = y.reshape(-1, max_length, output_size)
            ### Make it work for the nabsew dimensionality
            for sentence_index in range(len(y)):
                sentence = y[sentence_index]
                sentence_labels = sentence_batch[sentence_index]
                sentence_inputs = typo_batch[sentence_index]
                
                # Get predictions, labels, and inputs in a flattened form
                batch_preds = torch.argmax(sentence, dim=1).cpu().numpy().reshape(-1)
                batch_labels = sentence_labels.cpu().numpy().reshape(-1)
                batch_inputs = sentence_inputs.cpu().numpy().reshape(-1)
                
                # Append these results for later accuracy and F1 calculations
                preds.extend(batch_preds)
                labels.extend(batch_labels)
                inputs.extend(batch_inputs)
                # print("label_char", "pred_char", "input_char")
                
                # Process each element in the batch
                for label_idx in range(len(batch_preds)):
                    label_char, label_index_ = index_to_char(batch_labels[label_idx])
                    pred_char, pred_index_ = index_to_char(batch_preds[label_idx])
                    input_char, input_index_ = index_to_char(batch_inputs[label_idx],target = False)


                    # print(label_char, pred_char, input_char)
                    # Ignore padding characters
                    if label_char != "_":
                        word_label.append(label_char)
                    if pred_char != "_":
                        word_pred.append(pred_char)
                    if input_char != "_":
                        word_input.append(input_char)
                    
                    # If end of a word (detected by space), append word to csv_data
                    if label_char == " ":
                        # Join and append the completed word to csv data
                        csv_data.append(["".join(word_input), "".join(word_label), "".join(word_pred)])
                        # Clear the lists for the next word
                        word_label = []
                        word_pred = []
                        word_input = []
                csv_data.append(["".join(word_input), "".join(word_label), "".join(word_pred)])
                word_label = []
                word_pred = []
                word_input = []
            #     if sentence_index == 2:
            #         break
            # break

    # Write data to CSV file
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
    
    # Display a subset of rows in the old format for console output
    print(f"\nDisplaying the first {sample_size} rows from the output file:\n")
    print("Input Word      | Label Word      | Predicted Word")
    print("-" * 50)
    for row in csv_data[1:sample_size + 1]:  # Skip header row, take only `sample_size` rows
        print(f"{row[0]:<15} | {row[1]:<15} | {row[2]}")
    
    # Compute and display metrics
    loss_avg = loss / len(loader)
    print(f"\n{loader_str} Loss: {loss_avg:.4f}")
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')


    # # ADD EDIT DISTANCE METRIC HERE
    
    
    print(f"\n{loader_str} Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    # Calculate edit distance for each word and store in a list
    edit_distances = []
    for row in csv_data[1:]:  # Skip header row
        input_word = row[0]
        predicted_word = row[2]
        if input_word != "" and input_word != " ":
            try:
                edit_distances.append(editdistance.eval(input_word, predicted_word) / len(input_word))
            except:
                print(f"input_word '{input_word}'", "predicted_word", predicted_word)
    # Calculate average edit distance
    avg_edit_distance = sum(edit_distances) / len(edit_distances)
    print(f"\n{loader_str} Average Normalized Edit Distance: {avg_edit_distance:.4f}")

In [63]:
n_print = 200

In [64]:
# get_metrics(train_loader, model, "train", "train_predictions.csv", n_print)

In [65]:
get_metrics(validation_loader, model, "validation", "validation_predictions.csv", n_print)

  0%|          | 0/95 [00:00<?, ?it/s]


Displaying the first 200 rows from the output file:

Input Word      | Label Word      | Predicted Word
--------------------------------------------------
mike            | mike            | mile 
ncoihls         | nichols         | kichios 
won             | won             | won 
the             | the             | the 
acmeday         | academy         | academy 
aarwd           | award           | award 
for             | for             | for 
bset            | best            | best 
dotircer        | director        | directoraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
it              | it              | it 
was             | was             | was 
the             | the             | the 
frsit           | first           | first 
slb             | si              | sha
ia e            | able            | la e 
pnaenmret       | permanent       | prneement 
eligsnh         | english         | engiish 
stnelmeett      | sett

In [ ]:
# get_metrics(test_loader, model, "test", "test_predictions.csv", n_print)